In [53]:
import datetime
import Model
import numpy as np
import pandas as pd
import math
from multiprocessing import Pool
import os
import warnings

warnings.filterwarnings("once")


# positive numbers, start should be lower than end
# a rebalance index represents the month before the rebalancing takes place
# so returns are calculated starting at rebalanceIndex + 1
def rebalanceIndexes(startIndex, endIndex):
    indexes = list(range(startIndex, endIndex, 3))
    indexes.reverse()
    return (indexes)


# get ln returns for an equally balanced portfolio of stocks
def getReturns(portfolio, index, length):
	returns = 0
	print(portfolio)
	print(-1 * index + 1)
	print(-1 * index + length)
	nancount = 0
	for stock in portfolio:
		indReturn = Model.rateOfReturn(Model.retrieveData(stock, 'Last Price', -1 * index + 1, -1 * index + length, []))
		print(stock + ": " + str(indReturn))
		if (not math.isnan(indReturn)):
			returns += indReturn
		else:
			nancount += 1
	if len(portfolio) - nancount == 0:
		total = 0
	else:
		total = returns/(len(portfolio)-nancount)
	print("Total Return: " +str(total))
	return (total)


# make a portfolio with predicted probabilities higher than a hardcoded threshold
def makePortfolio(treeTuple):
	featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
	addedStocks, probabilities = Model.predict_probabilities(treeTuple[1], startIndex = -1 * treeTuple[0] - 11, endIndex = -1 * treeTuple[0], features = featureList, sector = "Financials")
	probabilityThreshold = 0.7
	stockTuples = zip(addedStocks, probabilities)
	stockTuples = list(filter(lambda x: x[1][1] > probabilityThreshold, stockTuples))
	if len(stockTuples) == 0:
		print("No portfolio, probabilities lower than threshold of " + str(probabilityThreshold))
		return 0
	stocks, probabilities = zip(*stockTuples)
	return(getReturns(stocks, treeTuple[0], 3))

In [57]:
rebalanceIndexes(4,100)

[97,
 94,
 91,
 88,
 85,
 82,
 79,
 76,
 73,
 70,
 67,
 64,
 61,
 58,
 55,
 52,
 49,
 46,
 43,
 40,
 37,
 34,
 31,
 28,
 25,
 22,
 19,
 16,
 13,
 10,
 7,
 4]

In [2]:
indexes = []
for i in rebalanceIndexes(4,100):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

32
[array([-312, -309, -306, -303, -300, -297, -294, -291, -288, -285, -282,
       -279, -276, -273, -270, -267, -264, -261, -258, -255, -252, -249,
       -246, -243, -240, -237, -234, -231, -228, -225, -222, -219, -216,
       -213, -210, -207, -204, -201, -198, -195, -192, -189, -186, -183,
       -180, -177, -174, -171, -168, -165, -162, -159, -156, -153, -150,
       -147, -144, -141, -138, -135, -132, -129, -126, -123, -120, -117,
       -114]), array([-309, -306, -303, -300, -297, -294, -291, -288, -285, -282, -279,
       -276, -273, -270, -267, -264, -261, -258, -255, -252, -249, -246,
       -243, -240, -237, -234, -231, -228, -225, -222, -219, -216, -213,
       -210, -207, -204, -201, -198, -195, -192, -189, -186, -183, -180,
       -177, -174, -171, -168, -165, -162, -159, -156, -153, -150, -147,
       -144, -141, -138, -135, -132, -129, -126, -123, -120, -117, -114,
       -111]), array([-306, -303, -300, -297, -294, -291, -288, -285, -282, -279, -276,
       -273, -270

In [3]:
def poolBuild(trainIndexes):
    featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
    forest = Model.buildWithIndexesTripleClass(modelType = Model.randomForestClassifier, indexes = trainIndexes, target= 'Rate of Return', features = featureList, \
                                               featureLength = 12, targetLength = 3, sector = "Financials", percentileTarget = 90, percentileAvoid = 10)
    return forest

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

# of multiprocess cpus: 8


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]
/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]
/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError 

Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]


Finished fitting.


...

Finished fitting.


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]
/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]


Finished fitting.


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]


Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.
Finished fitting.
[RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False), RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=Fal

In [52]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsList = pool.map(makePortfolio, zip(rebalanceIndexes(4,100), forestList))
print(returnsList)

# of multiprocess cpus: 8


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]


('AMP', 'BANC', 'EIG', 'PKBK', 'PRAA', 'WLTW')
-96
-94
AMP: 0.1468078454670687
BANC: 0.31334981920035876
EIG: 0.22344699540696444
PKBK: 0.339697432261274
PRAA: 0.21889641304060126
WLTW: 0.14567167583330587
Total Return: 0.23131169686826217


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]


('BANC',)
-93
-91
BANC: 0.039231524022529385
Total Return: 0.039231524022529385


...

/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]


('FBIZ', 'HCI', 'SFST')
Total Return: 0.08193632910494042
-78
-76
FBIZ: -0.005581409838195661
HCI: 0.23842704921443603
SFST: 0.012963347938580894


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]


('AGO', 'CNS', 'FNBG', 'HCI', 'ORC', 'PFBC', 'WETF')
Total Return: 0.159055468787111
-75
-73
AGO: 0.4693590916791104
CNS: 0.2156208371741566
FNBG: 0.09947198125146683
HCI: 0.19089094772315685
ORC: 0.09852752234763962
PFBC: 0.11969437932434923
WETF: -0.08017647799010263
('ANCX', 'BFIN', 'FBIZ', 'GBNK', 'HCI', 'HIIQ', 'HTH', 'PFBC', 'TREE', 'WSBF')
-72
-70
ANCX: 0.20258824159142286
BFIN: 0.164463939393948
FBIZ: 0.23498193706898407
GBNK: 0.20409535634351528
HCI: 0.25314760936256686
HIIQ: -0.06919185666321237
HTH: -0.038347308274623515
PFBC: 0.20677265265811906
TREE: 0.04604393850140687
WSBF: 0.2845402776546675
Total Return: 0.14890947876367946
('BFIN', 'GNW', 'HCI', 'HTH', 'LTXB', 'PRAA', 'TREE', 'UCFC', 'WSBF')
Total Return: 0.13217944666155015
-69
-67
BFIN: 0.14780950448881103
GNW: -0.03891541624967365
HCI: 0.28355267910976956
HTH: 0.010521378741532583
LTXB: 0.14360344955359539
PRAA: 0.20317974211150158
TREE: 0.3582241309255618
UCFC: 0.2523774870908523
WSBF: -0.17073793581799923
('BOFI'

ZeroDivisionError: division by zero

In [58]:
rebalanceIndexes(100,150)

[148,
 145,
 142,
 139,
 136,
 133,
 130,
 127,
 124,
 121,
 118,
 115,
 112,
 109,
 106,
 103,
 100]

In [59]:
indexes = []
for i in rebalanceIndexes(4,100):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

32
[array([-312, -309, -306, -303, -300, -297, -294, -291, -288, -285, -282,
       -279, -276, -273, -270, -267, -264, -261, -258, -255, -252, -249,
       -246, -243, -240, -237, -234, -231, -228, -225, -222, -219, -216,
       -213, -210, -207, -204, -201, -198, -195, -192, -189, -186, -183,
       -180, -177, -174, -171, -168, -165, -162, -159, -156, -153, -150,
       -147, -144, -141, -138, -135, -132, -129, -126, -123, -120, -117,
       -114]), array([-309, -306, -303, -300, -297, -294, -291, -288, -285, -282, -279,
       -276, -273, -270, -267, -264, -261, -258, -255, -252, -249, -246,
       -243, -240, -237, -234, -231, -228, -225, -222, -219, -216, -213,
       -210, -207, -204, -201, -198, -195, -192, -189, -186, -183, -180,
       -177, -174, -171, -168, -165, -162, -159, -156, -153, -150, -147,
       -144, -141, -138, -135, -132, -129, -126, -123, -120, -117, -114,
       -111]), array([-306, -303, -300, -297, -294, -291, -288, -285, -282, -279, -276,
       -273, -270

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

# of multiprocess cpus: 8


/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]
/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  values = data.loc[data.index[len(data) + startIndex:len(data) + endIndex + 1], features]
/Users/masontian/Documents/GitHub/Equity-Analysis/Model.py:168: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError 

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsList = pool.map(makePortfolio, zip(rebalanceIndexes(4,100), forestList))
print(returnsList)

In [60]:
rebalanceIndexes(150,200)

[198,
 195,
 192,
 189,
 186,
 183,
 180,
 177,
 174,
 171,
 168,
 165,
 162,
 159,
 156,
 153,
 150]